In [2]:
# pip install -U datasets

In [3]:
from pydantic.dataclasses import dataclass
import numpy as np

from datasets import load_dataset

In [4]:
@dataclass
class DataConfig:
  dataset_path_or_name: str = "sentence-transformers/natural-questions"
  test_size: float = 0.2

In [5]:
def load_data(cfg: DataConfig):
  data = load_dataset(cfg.dataset_path_or_name)['train']
  data = data.train_test_split(test_size=cfg.test_size, seed=1, shuffle=True)
  return data

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

def train_tfidf(data_train) -> TfidfVectorizer:
  vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=50000,
    min_df=5,
    max_df=0.8,
    sublinear_tf=True
  )

  vectorizer.fit(data_train['answer'])
  return vectorizer

In [ ]:
data = load_data(DataConfig())
model = train_tfidf(data_train=data['train'])

X_test = model.transform(data['test']['answer'])
# X_test

# Задача 1. Метрики


- Реализуйте функцию для расчёта метрики Recall@K. Эта метрика показывает, какая доля вопросов содержит в ТОП-K выдачи правильный документ. Функция должна вернуть число - значение метрики. Функция должна принимать:
1. target - ID правильного документа к каждому запросу;
2. predict - отсортированные по релевантности (метрике близости) ID документов к каждому запросу.

- По такому же принципу реализуйте функцию для расчёта метрики MRR.

In [8]:
def calculate_recall_single(target, candidates, K):
  return target in candidates[:K]

def calculate_recall(targets, candidates, K):
  return sum([calculate_recall_single(targets[i], candidates[i], K) for i in range(len(targets))]) / len(targets)

In [9]:
def calculate_mrr_single(target, candidates):
  return 1. / (np.where(candidates == target)[0] + 1)

def calculate_mrr(targets, candidates):
  return sum([calculate_mrr_single(targets[i], candidates[i]) for i in range(len(targets))]) / len(targets)

# Задача 2. TF-IDF Baseline

- Настройте алгоритм TF-IDF на обучающей выборке.
- При помощи уже настроенного TF-IDF векторизуйте вопросы и документы из тестовой выборки.
- При помощи метрики близости Cosine Similarity сделайте ранжирование документов из тестовой выборки для каждого вопроса из тестовой выборки.
- Рассчитайте метрики MRR и Recall@1, Recall@3, Recall@10.

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

def vectorize(model, data):
  vectors = model.transform(data)
  return vectors

def evaluate(model, data_test):
  doc_vectors = vectorize(model, data_test['answer'])
  query_vectors = vectorize(model, data_test['query'])

  mat = cosine_similarity(doc_vectors, query_vectors)
  sorted_mat = (-mat).argsort(axis=1)

  return {
      'Recall@1' : calculate_recall([i for i in range(len(data_test['answer']))], sorted_mat, 1),
      'Recall@3' : calculate_recall([i for i in range(len(data_test['answer']))], sorted_mat, 3),
      'Recall@10' : calculate_recall([i for i in range(len(data_test['answer']))], sorted_mat, 10),
      'MRR' : calculate_mrr([i for i in range(len(data_test['answer']))], sorted_mat)
  }


def find_similar_documents(query, documents, vectorizer, tfidf_matrix, top_n=3):
    # Vectorize the query
    query_vec = vectorizer.transform([query])

    # Compute similarity between query and all documents
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # Get top N most similar documents
    most_similar_indices = similarities.argsort()[::-1][:top_n]

    # Return results
    results = [(documents[i], similarities[i]) for i in most_similar_indices]
    return results


In [11]:
evaluate(model, data['test'])

{'Recall@1': 0.38609268219683746,
 'Recall@3': 0.5858731979847359,
 'Recall@10': 0.7548760413029381,
 'MRR': array([0.51256309])}

Вопросы:
- Какие получились метрики? Что можно о них сказать?

1. MRR (Mean Reciprocal Rank): на каком месте в среднем расположен ответ в формате дроби (1/2.63 = 0.38, значит где-то третье место)
2. Recall@K = n%: в n случаях ответ находится в топ K

- Какие ограничения есть у текущего подхода?
Игнорирование контекста, синонимов, не учитывает выбор людей